In [1]:
import torch
import torch.autograd as autograd         # computation graph
from torch import Tensor                  # tensor node in the computation graph
import torch.nn as nn                     # neural networks
import torch.optim as optim               # optimizers e.g. gradient descent, ADAM, etc.

import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.ticker
from torch.nn.parameter import Parameter
import matplotlib as mpl

import numpy as np
import time
#from pyDOE import lhs         #Latin Hypercube Sampling
import scipy.io
from scipy.io import savemat

from smt.sampling_methods import LHS

#Set default dtype to float32
torch.set_default_dtype(torch.float)

#PyTorch random number generator
torch.manual_seed(1234)

# Random number generators in other libraries
np.random.seed(1234)

# Device configuration
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

print(device)

if device == 'cuda': 
    print(torch.cuda.get_device_name())
    

cuda:0


In [2]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [3]:
# %cd '/content/gdrive/MyDrive/Virginia Tech /Fall 2022/Codes from GPU/PINN_Stan/2D Klein Gordon/stan'

In [4]:
# !pip install smt

In [5]:
lr_tune = np.array([0.05,0.1,0.25,0.5,1])

In [6]:
def true_2D_1(xt): #True function for 2D_1 Klein Gordon Equation x \in [-50,50] , t \in [0,10]
    y = xt[:,0]*np.cos(xt[:,1])
    return y.reshape(-1,1)    

In [7]:

loss_thresh = 0.01

x = np.linspace(-5,5,500).reshape(-1,1)
t = np.linspace(0,10,1000).reshape(-1,1)

X,T = np.meshgrid(x,t)

X = X.flatten('F').reshape(-1,1)
T = T.flatten('F').reshape(-1,1)
  
xt = np.hstack((X,T))

y_true = true_2D_1(xt)
y_true_norm = np.linalg.norm(y_true,2)

#bound_pts_idx = ((X == -5) + (X == 5) + (T == 0)).reshape(-1,)

#xt_bound = xt[bound_pts_idx,:]
#y_bound = y_true[bound_pts_idx,:]


xt_test_tensor = torch.from_numpy(xt).float().to(device)


lb_xt = xt[0]
ub_xt = xt[-1]

In [8]:
def trainingdata(N_I,N_B,N_f,seed):
    '''Boundary Conditions''' 
    
    np.random.seed(seed)
    x_BC1 = np.random.uniform(size = N_I).reshape(-1,1)
    t_BC1 = np.zeros((N_I,1))
    samples = np.hstack((x_BC1,t_BC1))
    xt_BC1 = lb_xt + (ub_xt - lb_xt)*samples
    y_BC1 = true_2D_1(xt_BC1)
    
    x_BC2 = np.zeros((int(N_B/2),1))
    t_BC2 = np.random.uniform(size = int(N_B/2)).reshape(-1,1)
    samples = np.hstack((x_BC2,t_BC2))
    xt_BC2 = lb_xt + (ub_xt - lb_xt)*samples
    y_BC2 = true_2D_1(xt_BC2)
    
    x_BC3 = np.ones((int(N_B/2),1))
    t_BC3 = np.random.uniform(size = int(N_B/2)).reshape(-1,1)
    samples = np.hstack((x_BC3,t_BC3))
    xt_BC3 = lb_xt + (ub_xt - lb_xt)*samples
    y_BC3 = true_2D_1(xt_BC3)

    xt_BC = np.vstack((xt_BC1,xt_BC2,xt_BC3))
    y_BC = np.vstack((y_BC1,y_BC2,y_BC3))

    '''Collocation Points'''

    # Latin Hypercube sampling for collocation points 
    # N_f sets of tuples(x,t)
    x01 = np.array([[0.0,1.0],[0.0,1.0]])
    sampling = LHS(xlimits=x01,random_state =seed)
    samples = sampling(N_f)
    xt_coll = lb_xt + (ub_xt - lb_xt)*samples
    
    xt_coll = np.vstack((xt_coll, xt_BC)) # append training points to collocation points 

    return xt_coll, xt_BC, y_BC

In [9]:
class Sequentialmodel(nn.Module):
    
    def __init__(self,layers):
        super().__init__() #call __init__ from parent class 
              
        'activation function'
        self.activation = nn.Sigmoid()

        'loss function'
        self.loss_function = nn.MSELoss(reduction ='mean')
        
        'Initialise neural network as a list using nn.Modulelist'  
        self.linears = nn.ModuleList([nn.Linear(layers[i], layers[i+1]) for i in range(len(layers)-1)])
        
        # std = gain * sqrt(2/(input_dim+output_dim))
        for i in range(len(layers)-1):
            nn.init.xavier_normal_(self.linears[i].weight.data, gain=1.0)
            # set biases to zero
            nn.init.zeros_(self.linears[i].bias.data)
            
        
        self.beta = Parameter(torch.ones((50,len(layers)-2)))
        self.beta.requiresGrad = True

            
    'foward pass'
    def forward(self,xt):
        if torch.is_tensor(xt) != True:         
            xt = torch.from_numpy(xt)                
        
        ubxt = torch.from_numpy(ub_xt).float().to(device)
        lbxt = torch.from_numpy(lb_xt).float().to(device)
    
                      
        #preprocessing input 
        xt = (xt - lbxt)/(ubxt - lbxt)
        
        #convert to float
        a = xt.float()
        
        for i in range(len(layers)-2):
            z = self.linears[i](a)
            a = z*self.activation(self.beta[:,i]*z)
            
        a = self.linears[-1](a) 
         
        return a
                        
    def loss_BC(self,xt,y):
                
        loss_bc = self.loss_function(self.forward(xt), y)
                
        return loss_bc
    
    def loss_PDE(self, xt_coll, f_hat):
        
        g = xt_coll.clone()             
        g.requires_grad = True
        y = self.forward(g) 
        
        
        y_x_t = autograd.grad(y,g,torch.ones([xt_coll.shape[0], 1]).to(device), retain_graph=True, create_graph=True,allow_unused = True)[0]
        
        y_xx_tt = autograd.grad(y_x_t,g,torch.ones(xt_coll.shape).to(device), create_graph=True,allow_unused = True)[0]

        #du_dx = u_x_t[:,[0]]
        
        d2y_dx2 = y_xx_tt[:,[0]]
        d2y_dt2 = y_xx_tt[:,[1]]    
        

        f = d2y_dt2 - d2y_dx2 + torch.pow(y,2) + (g[:,0]*torch.cos(g[:,1])).reshape(-1,1) - (torch.pow(g[:,0],2)*torch.pow(torch.cos(g[:,1]),2)).reshape(-1,1)
        
        loss_f = self.loss_function(f,f_hat)
                
        return loss_f
    
    def loss(self,xt_BC,y_BC,xt_coll,f_hat):

        loss_BC = self.loss_BC(xt_BC,y_BC)
        loss_f = self.loss_PDE(xt_coll,f_hat)
        
        loss_val = loss_BC + loss_f
        
        return loss_val
         
    'test neural network'
    def test(self):
        y_pred = self.forward(xt_test_tensor)
        y_pred = y_pred.cpu().detach().numpy()
   
        return y_pred
    
    def test_loss(self):
        y_pred = self.test()
        
        test_mse = np.mean(np.square(y_pred.reshape(-1,1) - y_true.reshape(-1,1)))
        test_re = np.linalg.norm(y_pred.reshape(-1,1) - y_true.reshape(-1,1),2)/y_true_norm
        
        return test_mse, test_re  

In [10]:
def data_update(loss_np):
    train_loss.append(loss_np)
    beta_val.append(PINN.beta.cpu().detach().numpy())
    
    test_mse, test_re = PINN.test_loss()
    test_mse_loss.append(test_mse)
    test_re_loss.append(test_re)

In [11]:
def train_step(xt_BC, y_BC, xt_coll, f_hat,seed):
    # x_coll_np_array = colloc_pts(N_f,seed*123)
    # x_coll_train = torch.from_numpy(x_coll_np_array).float().to(device)        
    
    # f_hat = torch.zeros(x_coll_train.shape[0],1).to(device)
    
    def closure():
        optimizer.zero_grad()
        loss = PINN.loss(xt_BC, y_BC, xt_coll,f_hat)
        loss.backward()
        #print(loss.cpu().detach().numpy())
        return loss

    optimizer.step(closure)

In [12]:
def train_model(max_iter,rep): 
  print(rep) 
  torch.manual_seed(rep*11)
  start_time = time.time() 
  thresh_flag = 0

  xt_coll, xt_BC, y_BC = trainingdata(N_I,N_B,N_f,rep*11)
  xt_coll = torch.from_numpy(xt_coll).float().to(device)
  xt_BC = torch.from_numpy(xt_BC).float().to(device)
  y_BC = torch.from_numpy(y_BC).float().to(device)

  f_hat = torch.zeros(xt_coll.shape[0],1).to(device)

  nan_flag = 0  
  for i in range(max_iter):
    train_step(xt_BC, y_BC, xt_coll,f_hat,i)
    
    loss_np = PINN.loss(xt_BC, y_BC, xt_coll,f_hat).cpu().detach().numpy()
    if(thresh_flag == 0):
        if(loss_np < loss_thresh):
            time_threshold[rep] = time.time() - start_time
            epoch_threshold[rep] = i+1            
            thresh_flag = 1       
    data_update(loss_np)
    
    print(i,"Train Loss",train_loss[-1],"Test MSE",test_mse_loss[-1],"Test RE",test_re_loss[-1])
    
    
    if(np.isnan(loss_np)):
            nan_flag =1
            print("NAN BREAK!")
            break
            
    

  elapsed_time[rep] = time.time() - start_time  
  print('Training time: %.2f' % (elapsed_time[rep]))
    
  return nan_flag

In [13]:
nan_tune = []
for tune_reps in range(5):
#for tune_reps in range(4,5):
  max_reps = 10 #10
  max_iter = 100 #100
  label = "KG_swish_tune"+str(tune_reps)

  train_loss_full = []
  test_mse_full = []
  test_re_full = []
  beta_full = []
  elapsed_time= np.zeros((max_reps,1))
  time_threshold = np.empty((max_reps,1))
  time_threshold[:] = np.nan
  epoch_threshold = max_iter*np.ones((max_reps,1))

  N_I = 200  #Total number of data points for 'y'
  N_B = 400
  N_f = 10000 #Total number of collocation points

  for reps in range(max_reps):
      print(reps)
      print(label)

      train_loss = []
      test_mse_loss = []
      test_re_loss = []
      beta_val = []
      
      torch.manual_seed(reps*36)

      layers = np.array([2,50,50,50,50,50,50,50,50,50,1]) #9 hidden layers

      PINN = Sequentialmodel(layers)
    
      PINN.to(device)

      'Neural Network Summary'
      print(PINN)

      params = list(PINN.parameters())
      

      optimizer = torch.optim.LBFGS(PINN.parameters(), lr=lr_tune[tune_reps], 
                                max_iter = 10, 
                                max_eval = 15, 
                                tolerance_grad = 1e-8, 
                                tolerance_change = 1e-8, 
                                history_size = 100, 
                                line_search_fn = 'strong_wolfe')



          
      nan_flag = train_model(max_iter,reps)
    
      torch.save(PINN.state_dict(),label+'_'+str(reps)+'.pt')
      train_loss_full.append(train_loss)
      test_mse_full.append(test_mse_loss)
      test_re_full.append(test_re_loss)
      #elapsed_time[reps] = time.time() - start_time
      beta_full.append(beta_val)
    
      if(nan_flag == 1):
            nan_tune.append(tune_reps)
            break
            
      #print('Training time: %.2f' % (elapsed_time[reps]))
      
  mdic = {"train_loss": train_loss_full,"test_mse_loss": test_mse_full,"test_re_loss": test_re_full,"Time": elapsed_time, "beta": beta_full, "label": label}
  savemat(label+'.mat', mdic)

0
Sequentialmodel(
  (activation): Sigmoid()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=2, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): Linear(in_features=50, out_features=1, bias=True)
  )
)
0
0 Train Loss 58.679802 Test MSE 8.713152247558138 Test RE 1.4108983046381993
1 Train Loss 58.223907 Test MSE 8.39443530988614 Test RE 1.3848534063869096
2 Train Loss 58.10724 Test MSE 8.678306331339545 Test RE 1.4080742235331103
3 Train Loss 58.076767 Test MSE 8.811811303587694 Test RE 1.418863

97 Train Loss 19.796864 Test MSE 4.057734296745274 Test RE 0.9628305531773542
98 Train Loss 19.522972 Test MSE 4.100779175912717 Test RE 0.9679239856075097
99 Train Loss 19.409264 Test MSE 4.148200193208979 Test RE 0.9735043892591039
Training time: 85.07
1
Sequentialmodel(
  (activation): Sigmoid()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=2, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): Linear(in_features=50, out_features=1, bias=True)
  )
)
1
0 Train Loss 56.888855 Test MSE 8.714103

94 Train Loss 13.278489 Test MSE 4.055132727934569 Test RE 0.9625218499239264
95 Train Loss 13.055972 Test MSE 3.9695743153493375 Test RE 0.9523136925028943
96 Train Loss 13.016237 Test MSE 3.9524166623562986 Test RE 0.9502533756000096
97 Train Loss 12.995108 Test MSE 3.9907749302021713 Test RE 0.9548533540469673
98 Train Loss 12.964328 Test MSE 3.9948574337400014 Test RE 0.9553416296106836
99 Train Loss 12.878113 Test MSE 3.9682759660193367 Test RE 0.9521579406684663
Training time: 84.12
2
Sequentialmodel(
  (activation): Sigmoid()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=2, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50,

91 Train Loss 43.997948 Test MSE 8.627704912776938 Test RE 1.4039631264962542
92 Train Loss 43.997948 Test MSE 8.627704912776938 Test RE 1.4039631264962542
93 Train Loss 43.997948 Test MSE 8.627704912776938 Test RE 1.4039631264962542
94 Train Loss 43.997948 Test MSE 8.627704912776938 Test RE 1.4039631264962542
95 Train Loss 43.997948 Test MSE 8.627704912776938 Test RE 1.4039631264962542
96 Train Loss 43.997948 Test MSE 8.627704912776938 Test RE 1.4039631264962542
97 Train Loss 43.997948 Test MSE 8.627704912776938 Test RE 1.4039631264962542
98 Train Loss 43.997948 Test MSE 8.627704912776938 Test RE 1.4039631264962542
99 Train Loss 43.997948 Test MSE 8.627704912776938 Test RE 1.4039631264962542
Training time: 38.12
3
Sequentialmodel(
  (activation): Sigmoid()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=2, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)

88 Train Loss 10.035139 Test MSE 3.8237811261740013 Test RE 0.9346619713783855
89 Train Loss 9.864943 Test MSE 3.8186492750740726 Test RE 0.9340345614488891
90 Train Loss 9.771688 Test MSE 3.8399658875807563 Test RE 0.9366379353700242
91 Train Loss 9.719086 Test MSE 3.8123719701108407 Test RE 0.933266537087535
92 Train Loss 9.664425 Test MSE 3.7726484630014334 Test RE 0.9283916590091879
93 Train Loss 9.605186 Test MSE 3.774854964457194 Test RE 0.9286631126218937
94 Train Loss 9.507164 Test MSE 3.827023222678767 Test RE 0.9350581266707251
95 Train Loss 9.399088 Test MSE 3.7821789346961383 Test RE 0.9295635717379208
96 Train Loss 9.37146 Test MSE 3.833519489208957 Test RE 0.9358514078801964
97 Train Loss 9.344646 Test MSE 3.833690601352725 Test RE 0.9358722938772808
98 Train Loss 9.28567 Test MSE 3.7559743637813208 Test RE 0.92633776541635
99 Train Loss 9.179633 Test MSE 3.7890898694775617 Test RE 0.9304124503072381
Training time: 84.57
4
Sequentialmodel(
  (activation): Sigmoid()
  (los

...

0 Train Loss 56.87461 Test MSE 8.581138204485503 Test RE 1.4001691628789386
1 Train Loss 56.510048 Test MSE 8.720739409961148 Test RE 1.4115124558054222
2 Train Loss 55.306023 Test MSE 7.000399765538237 Test RE 1.2646474745028435
3 Train Loss 46.897064 Test MSE 8.022291359912911 Test RE 1.353808622342108
4 Train Loss 43.933952 Test MSE 8.322723839837876 Test RE 1.3789254984763264
5 Train Loss 43.419624 Test MSE 8.153486501073896 Test RE 1.3648337035655964
6 Train Loss 43.137188 Test MSE 8.357389448922364 Test RE 1.3817942480227354
7 Train Loss 42.849068 Test MSE 8.16670070533522 Test RE 1.365939236145416
8 Train Loss 42.847622 Test MSE 8.163047204683298 Test RE 1.3656336648789593
9 Train Loss 42.822235 Test MSE 8.162111629144539 Test RE 1.3655554042717297
10 Train Loss 42.821384 Test MSE 8.16408887923607 Test RE 1.3657207953598998
11 Train Loss 42.80039 Test MSE 8.179888221632508 Test RE 1.3670416446029332
12 Train Loss 42.771904 Test MSE 8.207051213385892 Test RE 1.3693095340966492
13

0 Train Loss 56.40176 Test MSE 8.592981953541514 Test RE 1.4011350915046237
1 Train Loss 56.192154 Test MSE 8.679482567615056 Test RE 1.4081696437568632
2 Train Loss 53.79621 Test MSE 8.379212443903933 Test RE 1.3835971572631283
3 Train Loss 52.882675 Test MSE 8.838661188715687 Test RE 1.421023642113924
4 Train Loss 50.04314 Test MSE 8.800640679123404 Test RE 1.417964000551021
5 Train Loss 43.06001 Test MSE 8.329837090187961 Test RE 1.3795146413795714
6 Train Loss 42.718376 Test MSE 8.512988586399361 Test RE 1.39459815347942
7 Train Loss 42.485466 Test MSE 8.483370933128716 Test RE 1.3921700571223394
8 Train Loss 42.404205 Test MSE 8.473513166124793 Test RE 1.3913609638239945
9 Train Loss 42.248894 Test MSE 8.40721272876658 Test RE 1.3859069688400967
10 Train Loss 41.697666 Test MSE 8.451522345781756 Test RE 1.3895543314416081
11 Train Loss 39.689636 Test MSE 8.066725658639088 Test RE 1.357552718994828
12 Train Loss 38.67398 Test MSE 7.787843132531532 Test RE 1.333879683595593
13 Train

0 Train Loss 56.55311 Test MSE 8.806875082687442 Test RE 1.4184661568766517
1 Train Loss 55.879303 Test MSE 8.690302077350593 Test RE 1.4090470555119454
2 Train Loss 51.958557 Test MSE 8.48790832845752 Test RE 1.392542313713446
3 Train Loss 46.980095 Test MSE 8.76811614894929 Test RE 1.41534139023963
4 Train Loss 42.97379 Test MSE 8.316648034084867 Test RE 1.3784220807701053
5 Train Loss 42.82012 Test MSE 8.340340075589728 Test RE 1.380384073614825
6 Train Loss 42.706512 Test MSE 8.495180760463334 Test RE 1.3931387504674364
7 Train Loss 42.58979 Test MSE 8.424971312284983 Test RE 1.3873699246549103
8 Train Loss 42.55623 Test MSE 8.527900883543081 Test RE 1.3958190856104569
9 Train Loss 42.39845 Test MSE 8.471637750682218 Test RE 1.391206982577728
10 Train Loss 42.124107 Test MSE 8.40347684687797 Test RE 1.3855990094589614
11 Train Loss 41.256115 Test MSE 8.087889891481774 Test RE 1.3593324213409388
12 Train Loss 38.943916 Test MSE 7.38168545010148 Test RE 1.2986311852520098
13 Train Lo

0 Train Loss 55.768982 Test MSE 8.648421960749662 Test RE 1.405647730466231
1 Train Loss 55.65703 Test MSE 8.490633507189427 Test RE 1.3927658447645124
2 Train Loss 55.36726 Test MSE 8.743636687868364 Test RE 1.4133642830673143
3 Train Loss 54.04268 Test MSE 8.53399634123417 Test RE 1.3963178388501927
4 Train Loss 45.491173 Test MSE 8.673286643703213 Test RE 1.4076669369685126
5 Train Loss 43.83725 Test MSE 8.598055244592397 Test RE 1.401548645104244
6 Train Loss 43.63657 Test MSE 8.831234891086376 Test RE 1.4204265402523841
7 Train Loss 43.414757 Test MSE 8.72536945653648 Test RE 1.4118871087384872
8 Train Loss 43.273224 Test MSE 8.651547648529792 Test RE 1.4059017200983241
9 Train Loss 43.256992 Test MSE 8.627286081207872 Test RE 1.4039290484174543
10 Train Loss 43.240074 Test MSE 8.653613301384924 Test RE 1.4060695473873843
11 Train Loss 43.193897 Test MSE 8.653191735973564 Test RE 1.4060352982543505
12 Train Loss 43.094986 Test MSE 8.662804717622961 Test RE 1.406816076039776
13 Tra

0 Train Loss 56.936543 Test MSE 8.701629490070056 Test RE 1.4099650707731648
1 Train Loss nan Test MSE nan Test RE nan
NAN BREAK!
Training time: 2.27
0
Sequentialmodel(
  (activation): Sigmoid()
  (loss_function): MSELoss()
  (linears): ModuleList(
    (0): Linear(in_features=2, out_features=50, bias=True)
    (1): Linear(in_features=50, out_features=50, bias=True)
    (2): Linear(in_features=50, out_features=50, bias=True)
    (3): Linear(in_features=50, out_features=50, bias=True)
    (4): Linear(in_features=50, out_features=50, bias=True)
    (5): Linear(in_features=50, out_features=50, bias=True)
    (6): Linear(in_features=50, out_features=50, bias=True)
    (7): Linear(in_features=50, out_features=50, bias=True)
    (8): Linear(in_features=50, out_features=50, bias=True)
    (9): Linear(in_features=50, out_features=1, bias=True)
  )
)
0
0 Train Loss 58.67505 Test MSE 8.694732515149182 Test RE 1.4094061857141251
1 Train Loss nan Test MSE nan Test RE nan
NAN BREAK!
Training time: 2

In [14]:
import scipy.io as sio
import numpy as np

In [20]:
for tune_reps in range(5):
    label = "KG_swish_tune"+str(tune_reps)+".mat"
    data = sio.loadmat(label)
 
    re = np.array(data["test_re_loss"])
    print("tune_reps"," ",np.mean(re[:,-1]))

tune_reps   1.0808060463897486
tune_reps   0.8801059752546545
tune_reps   [[1.41093885 1.41320998 1.44207193 1.32377782        nan]]
tune_reps   [[1.40996507        nan]]
tune_reps   nan


In [16]:
# tune_reps = 4
# label = "KG_swish_tune"+str(tune_reps)+".mat"
# data = sio.loadmat(label)
# for k in range(10):
#     print(tune_reps," ",data["test_re_loss"][0][k].shape)

4   ()
4   ()


IndexError: index 2 is out of bounds for axis 0 with size 2

In [ ]:
np.mean(data["test_re_loss"][:,-1])